In [ ]:
!git clone https://github.com/GKalliatakis/Keras-VGG16-places365.git
%cd Keras-VGG16-places365

In [ ]:
from google.colab import drive
import zipfile
import os

# Montar Google Drive
drive.mount('/content/drive')

# Rutas archivo ZIP
zips = [
    "/content/drive/MyDrive/TFG_Instagram/fotos_filtradas_LPGA.zip"
]

# Carpeta de destino base en /content
base_destino = "/content/"
os.makedirs(base_destino, exist_ok=True)

# Extraer cada ZIP en su propia subcarpeta
for zip_path in zips:
    nombre = os.path.splitext(os.path.basename(zip_path))[0]
    destino = os.path.join(base_destino, nombre)
    os.makedirs(destino, exist_ok=True)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(destino)
        print("Extraído")

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import urllib.request
from vgg16_places_365 import VGG16_Places365
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from google.colab import drive

drive.mount('/content/drive')

# Modelo preentrenado en Places365
model = VGG16_Places365(weights='places')

# Descargar categorías
file_name = 'categories_places365.txt'
if not os.path.exists(file_name):
    synset_url = 'https://raw.githubusercontent.com/GKalliatakis/Keras-VGG16-places365/master/categories_places365.txt'
    urllib.request.urlretrieve(synset_url, file_name)

# Leer las clases
with open(file_name) as f:
    classes = [line.strip().split(' ')[0][3:] for line in f]

# Ruta a la carpeta de imágenes
bbdd_path = "/content/fotos_filtradas_LPGA"

# Procesar todas las imágenes (top 5 escenas)
resultados = []
for archivo in tqdm(os.listdir(bbdd_path), desc="Procesando imágenes"):
    if archivo.lower().endswith((".jpg")):
        try:
            img_path = os.path.join(bbdd_path, archivo)
            image = Image.open(img_path).convert('RGB')
            image = image.resize((224, 224))
            image = np.array(image, dtype=np.uint8)
            image = np.expand_dims(image, 0)
            image = preprocess_input(image)

            preds = model.predict(image)[0]
            top_preds = np.argsort(preds)[::-1][:5]

            resultado = {
                "imagen": archivo,
                "escena_1": classes[top_preds[0]],
                "score_1": float(preds[top_preds[0]]),
                "escena_2": classes[top_preds[1]],
                "score_2": float(preds[top_preds[1]]),
                "escena_3": classes[top_preds[2]],
                "score_3": float(preds[top_preds[2]]),
                "escena_4": classes[top_preds[3]],
                "score_4": float(preds[top_preds[3]]),
                "escena_5": classes[top_preds[4]],
                "score_5": float(preds[top_preds[4]]),
            }
            resultados.append(resultado)
        except Exception as e:
            print(f"Error con {archivo}: {e}")

# Crear y guardar el DataFrame
df_escenas = pd.DataFrame(resultados)
#output_path = "/content/drive/MyDrive/TFG_Instagram/escenas_places365_top5_LPGA.csv"
#df_escenas.to_csv(output_path, sep=';', index=False)
print(f"Archivo guardado en Drive: {output_path}")


In [ ]:
import os
import urllib.request
import numpy as np
from PIL import Image
from vgg16_places_365 import VGG16_Places365
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import matplotlib.pyplot as plt # <-- 1. Importar la biblioteca

# Ruta a tu imagen cargada
image_path = '/content/fotos_filtradas_LPGA/scottie.scheffler_2024-04-15_18-52-44_UTC_3.jpg'

# Abrir y preparar la imagen
img_pil = Image.open(image_path).convert('RGB')
img_pil = img_pil.resize((224, 224))
img_array = np.array(img_pil, dtype=np.uint8)


plt.imshow(img_array)
plt.title("Imagen a clasificar")
plt.axis('off')
plt.show()


# Preparar la imagen para el modelo (añadir dimensión de batch y preprocesar)
image_for_model = np.expand_dims(img_array, 0)
image_for_model = preprocess_input(image_for_model)

# Cargar modelo
model = VGG16_Places365(weights='places')

# Predecir
predictions_to_return = 10
preds = model.predict(image_for_model)[0]
top_preds = np.argsort(preds)[::-1][:predictions_to_return]

# Descargar clases si no están
file_name = 'categories_places365.txt'
if not os.path.exists(file_name):
    synset_url = 'https://raw.githubusercontent.com/GKalliatakis/Keras-VGG16-places365/master/categories_places365.txt'
    urllib.request.urlretrieve(synset_url, file_name)

# Leer clases
with open(file_name) as f:
    classes = [line.strip().split(' ')[0][3:] for line in f]

# Mostrar predicciones
print('--SCENE CATEGORIES:')
for i in top_preds:
    print(f'{classes[i]}: {preds[i]:.4f}')


